In [33]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random
import pickle
import time
import datetime
import xlsxwriter

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set api token 
QX_TOKEN = "96c86855a5c381875a4ea3a956a7deafbbbbb82e5912ec73655c7c444d2597a7364eda2fa558e4e37f672640029443216899825f945d5a0684e31b796837ce61"
QX_URL = "https://quantumexperience.ng.bluemix.net/api"


#register token and url
register(QX_TOKEN, QX_URL)

backend = 'ibmqx5'

# add phase gate 
def addPhase(qc):
    qc.h(qr[perm[0]])
    qc.h(qr[perm[1]])
    swap(perm,0,1)
    qc.s(qr[perm[0]]) 
    qc.barrier(qr)
    
# remove phase gate
def removePhase(qc):
    qc.sdg(qr[perm[0]])
    swap(perm,0,1)
    qc.h(qr[perm[1]])
    qc.h(qr[perm[0]])
    qc.barrier(qr)

# compile into different states
def compilation(qc,i,perm):
    if i == 0: 
        qc.iden(qr[perm[0]])
        qc.iden(qr[perm[1]])
    elif i == 1:
        qc.x(qr[perm[0]])
        qc.iden(qr[perm[1]])
    elif i == 2:
        qc.iden(qr[perm[0]])
        qc.x(qr[perm[1]])
    elif i == 3:
        qc.x(qr[perm[0]])
        qc.x(qr[perm[1]])
    qc.barrier(qr)
        
            
# parse a string of gens
def addCliffGens(qc, string,perm):
    for char in string:
        i = int(char)
        if i==0:
            qc.x(qr[perm[0]])
        elif i==1:
            qc.x(qr[perm[1]])
        elif i==2:
            qc.z(qr[perm[0]])
        elif i==3:
            qc.z(qr[perm[1]])
        elif i ==4:
            qc.h(qr[perm[0]])
            qc.h(qr[perm[1]])
            swap(perm,0,1)
        elif i == 5:
            qc.cz(qr[perm[0]],qr[perm[1]])
            qc.z(qr[perm[0]])
            qc.z(qr[perm[1]])
        elif i==6:
            qc.cx(qr[perm[0]],qr[perm[1]])
        elif i==7:
            qc.cx(qr[perm[1]],qr[perm[0]])
            
# search dictionary for key corresponding to matrix
def findGenString(mat):
    for key, value in signedGroup.items():
        if np.array_equal(value, mat):
            return key
    print("OH NO")
    return None
        
# invert cliff gens
def invertCliffGens(qc, genString,perm):
    for char in genString[::-1]:
        i = int(char)
        if i==0:
            qc.x(qr[perm[0]])
        elif i==1:
            qc.x(qr[perm[1]])
        elif i==2:
            qc.z(qr[perm[0]])
        elif i==3:
            qc.z(qr[perm[1]])
        elif i ==4:
            swap(perm,0,1)
            qc.h(qr[perm[0]])
            qc.h(qr[perm[1]])
        elif i == 5:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[1]])
            qc.cz(qr[perm[0]],qr[perm[1]])
        elif i==6:
            qc.cx(qr[perm[0]],qr[perm[1]])
        elif i==7:
            qc.cx(qr[perm[1]],qr[perm[0]])
    qc.barrier(qr)

# add a random real clifford            
def addRandCliff(qc, mat,perm):
    genString = random.choice(list(r2group.keys()))
    addCliffGens(qc,genString,perm)
    qc.barrier(qr)
    return np.matmul(mat,r2group[genString])

# converts 0-3 to binary strings
def toBinString(num):
    if num ==0:
        return "00"
    elif num == 1:
        return "01"
    elif num == 2:
        return "10"
    elif num == 3:
        return "11"
    else:
        print("OH NO")


def writeToExcel(results,key,initialState,row,col):
    states = ["00","01","10","11"]

    worksheet.write(row,0,initialState)
    worksheet.write(row,1,key)
    col = 2
    for state in states:
        try:
            worksheet.write(row,col,results[state])
            
        except:
            print(state+"does not exist")
            worksheet.write(row,col,0)
        col = col+1
    
    return row+1

# used for reindexing qubits
def swap(perm, a, b):
    temp = perm[a]
    perm[a] = perm[b]
    perm[b] = temp

# all the important settings are here    
lengths = [3]
reps = [9,9,9,9]
initialStates = [0,1,2,3]
shots = 1024
phased = True
shouldWrite = True

device = get_backend("ibmqx5")
coupling_map = device.configuration['coupling_map']


pickle_in = open("r2group.pkl", "rb")
r2group = pickle.load(pickle_in)
pickle_in2 = open("r2group.pickle", "rb")
signedGroup = pickle.load(pickle_in2)

if shouldWrite:
    
    workbook = xlsxwriter.Workbook("uncoded_("+
                                   ",".join(map(str,lengths))+
                                   ")_("+
                                   ",".join(map(str,reps))+
                                   ")_"+
                                   str(int(phased))+
                                   "_"+
                                   str(datetime.datetime.now().strftime('%Y%m%d_%H%M%S'))+
                                   ".xlsx")
    worksheet = workbook.add_worksheet()
    
    states = ["Key","00","01","10","11"]
    for index in range(len(states)):
        worksheet.write(0,index+1,states[index])
    row = 1
    col = 1
    

startTime = time.time()

for length in lengths:

    for compiler in initialStates:
        
        print(str(length)+" "+str(compiler)+"\n")
        
        if shouldWrite:
            worksheet.write(row,0, "Length: "+str(length))
            row = row+1

        # run our trials
        for i in range(reps[compiler]):

            # run one trial of given length 
            mat = r2group[""]
            n = 2
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
            qc = QuantumCircuit(qr, cr)

            perm = {0: 0, 1: 1}
            
            # optional: add phase
            if phased:
                addPhase(qc)

            # add the first m-1 clifford gates
            for j in range(length-1):
                mat = addRandCliff(qc,mat,perm)
            
            #Find String
            genString = findGenString(mat)

            # invert everything
            invertCliffGens(qc,genString,perm)
            
            # optional: remove phase
            if phased:
                removePhase(qc)

            # get the qc compiled into one of the 4 different states
            compilation(qc,compiler,perm)
            
            # measure
            for i in range(n):
                qc.measure(qr[perm[i]], cr[i])
            
            #initial_layout = { ('q', 0): ('q', 12), ('q', 1): ('q', 5)}
            print(perm)
            print("got")
            job = execute(qc, backend, shots = shots)
            print("here")
            result = job.result()
            print("nice")
            data = result.get_counts(qc)
            
            # outdated
            #print(data)

            # plot_histogram(data)
            
            # set shouldWrite to false if running on simulator or don't actually care
            if shouldWrite:
                row = writeToExcel(data,genString,compiler,row,col)
            
            # optional: get success rate
            desired = toBinString(compiler)
            print(data[desired]/1024)
            
endTime = time.time()
print("Time elapsed: ", endTime-startTime)

if shouldWrite:
    workbook.close()
    

 
    
    

Qconfig loaded from C:\Users\ddrsq\OneDrive\Quantum\quantum_rb\Qconfig.py.
3 0

{0: 1, 1: 0}
got
here


Got a 403 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=CFvzYpiEf7IIHbHQEFkza7kpVZRhDLF5M31KyqFq8nke72Pif5SoOUlBQA0oOG7I: {"error":{"status":403,"message":"Your credits are not enough. 3 credits are needed to launch this job but you only have 0 credits remaining.","code":"MAX_CREDITS_EXCEEDED","statusCode":403}}
Got a 403 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=CFvzYpiEf7IIHbHQEFkza7kpVZRhDLF5M31KyqFq8nke72Pif5SoOUlBQA0oOG7I: {"error":{"status":403,"message":"Your credits are not enough. 3 credits are needed to launch this job but you only have 0 credits remaining.","code":"MAX_CREDITS_EXCEEDED","statusCode":403}}
Got a 403 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=CFvzYpiEf7IIHbHQEFkza7kpVZRhDLF5M31KyqFq8nke72Pif5SoOUlBQA0oOG7I: {"error":{"status":403,"message":"Your credits are not enough. 3 credits are needed to launch this job but you only have 0 credits remaining.","code

nice


QISKitError: '"{\'status\': 403, \'message\': \'Your credits are not enough. 3 credits are needed to launch this job but you only have 0 credits remaining.\', \'code\': \'MAX_CREDITS_EXCEEDED\', \'statusCode\': 403}"'